In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras

In [3]:
model = keras.models.load_model('../../models/tensorflow/NER_model_updated.h5')
model.summary()

Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2022-09-26 08:43:22.504602: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-09-26 08:43:22.505265: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 300)]             0         
                                                                 
 embedding (Embedding)       (None, 300, 50)           468000    
                                                                 
 dropout (Dropout)           (None, 300, 50)           0         
                                                                 
 bidirectional (Bidirectiona  (None, 300, 400)         401600    
 l)                                                              
                                                                 
 time_distributed (TimeDistr  (None, 300, 8)           3208      
 ibuted)                                                         
                                                                 
Total params: 872,808
Trainable params: 872,808
Non-trainable

# Pipeline
This is an example of a full pipeline of a sentence that needs to be predicted.

Steps:
1. Apply pre-processing (replace the dots and the commas)
2. Split the words
3. Import the embeddings (created from the dataset)
4. Pad the sequence
5. Predict
6. ???
7. Profit

In [5]:
import re

def remove_dot(x):
    pattern = '\w\.' # removes dots at the end of sentences... ex: "we went there." -> "we went there"
    finds = re.findall(pattern, x)
    for find in finds:
        x = re.sub(pattern, find[:-1], x)

    pattern_2 = '\.\w' # removes dots at the end of sentences. ex: "we went .there" -> "we went there"
    x = re.sub(pattern_2, '', x)

    pattern_3 = '\s\.\s' # removes dots at the end of sentences. ex: "abc . abc" -> "abc abc"
    x = re.sub(pattern_3, '', x)

    return x

In [15]:
# Sentence to predict
# sentence = 'EXW QTY:    192 UNITS CONCENTRATE 7-UP, AS PER BENEFICIARY\'S SALES QUOTATION NO. 790005454 DATED 02.01.2020'
sentence = '00,000 Bottles ( +/- 05/05 %) of First Class Technical Machinery Soda, packed in blue crates labelled \'Test Machine London\' Unit Price: Usd 5,00 DELIVERY TERMS; CIF TORONTO HS    Code:    55678.334    '

sentence = sentence.replace('_x005F_x000D_','')
sentence = sentence.replace('\n',' ')
sentence = sentence.replace(':',' ')
sentence = sentence.replace('+)','')
sentence = re.sub('\d-', '', sentence)
sentence = re.sub('\(\d\)', '', sentence)
sentence = re.sub('\\d\)', '', sentence)
sentence = re.sub(',\s', '', sentence)
sentence = re.sub(',\w', '', sentence)
sentence = remove_dot(sentence)
sentence = sentence.replace(' - ','')
sentence = sentence.replace(',','')
sentence = sentence.replace('++++++++++++++++++++++++++++++++++++','')
sentence = sentence.strip()

words = sentence.split()

import pickle
# Save the embeddings
with open("../../models/embeddings/words_embeddings", "rb") as fp: 
    word2idx = pickle.load(fp)


# Embedding
X = []
for word in words:
    if word in word2idx.keys():
        X.append(word2idx[word])
    else:
        X.append(9359)
X

max_len = 300 # taken from ner_nn.ipynb
from keras_preprocessing.sequence import pad_sequences
X = pad_sequences(maxlen=max_len, sequences=[X], padding="post", value=len(word2idx)-1)

# # Apply the predict function
output = model.predict(X)

with open("../../models/embeddings/tags_embeddings", "rb") as fp: 
    tag2idx = pickle.load(fp)
tag2idx = list(tag2idx)
tag2idx

predicted_output = []
for word in output[0]:
    predicted_output.append(tag2idx[word.argmax()])

for i in range(len(words)):
    print('Original word: ', words[i], '=== Predicted Tag: ', predicted_output[i])
# predicted_output

1/1 [==============================] - 6s 6s/step
Original word:  0000 === Predicted Tag:  O
Original word:  Bottles === Predicted Tag:  O
Original word:  ( === Predicted Tag:  O
Original word:  +/- === Predicted Tag:  Incoterms
Original word:  05/05 === Predicted Tag:  O
Original word:  %) === Predicted Tag:  O
Original word:  of === Predicted Tag:  O
Original word:  First === Predicted Tag:  O
Original word:  Class === Predicted Tag:  O
Original word:  Technical === Predicted Tag:  O
Original word:  Machinery === Predicted Tag:  O
Original word:  Sodapacked === Predicted Tag:  O
Original word:  in === Predicted Tag:  O
Original word:  blue === Predicted Tag:  O
Original word:  crates === Predicted Tag:  O
Original word:  labelled === Predicted Tag:  O
Original word:  'Test === Predicted Tag:  O
Original word:  Machine === Predicted Tag:  O
Original word:  London' === Predicted Tag:  O
Original word:  Unit === Predicted Tag:  O
Original word:  Price === Predicted Tag:  O
Original word

In [22]:
X

array([[9359, 9359, 2955,  586, 9359, 9359, 9359, 9359, 9359, 9359, 9359,
        9359, 6139, 9359, 9359, 9359, 9359, 9359, 9359, 9359, 9359, 9359,
        3630, 4046, 9359, 4270, 9359, 6000, 9359, 9359, 9359, 9359, 9359,
        9359, 9359, 9359, 9359, 9359, 9359, 9359, 9359, 9359, 9359, 9359,
        9359, 9359, 9359, 9359, 9359, 9359, 9359, 9359, 9359, 9359, 9359,
        9359, 9359, 9359, 9359, 9359, 9359, 9359, 9359, 9359, 9359, 9359,
        9359, 9359, 9359, 9359, 9359, 9359, 9359, 9359, 9359, 9359, 9359,
        9359, 9359, 9359, 9359, 9359, 9359, 9359, 9359, 9359, 9359, 9359,
        9359, 9359, 9359, 9359, 9359, 9359, 9359, 9359, 9359, 9359, 9359,
        9359, 9359, 9359, 9359, 9359, 9359, 9359, 9359, 9359, 9359, 9359,
        9359, 9359, 9359, 9359, 9359, 9359, 9359, 9359, 9359, 9359, 9359,
        9359, 9359, 9359, 9359, 9359, 9359, 9359, 9359, 9359, 9359, 9359,
        9359, 9359, 9359, 9359, 9359, 9359, 9359, 9359, 9359, 9359, 9359,
        9359, 9359, 9359, 9359, 9359, 

In [17]:
dictionar = {}
for key in tag2idx:
    dictionar[key] = []
dictionar

{'Quantity': [],
 'UnitPriceAmount': [],
 'O': [],
 'GoodsDescription': [],
 'Incoterms': [],
 'GoodsOrigin': [],
 'Tolerance': [],
 'HSCode': []}

In [18]:
y_pred = []
for word in output[0]:
    y_pred.append(tag2idx[word.argmax()])

for word, prediction in zip(words, y_pred):
    dictionar[prediction].append(word)
dictionar

{'Quantity': [],
 'UnitPriceAmount': [],
 'O': ['0000',
  'Bottles',
  '(',
  '05/05',
  '%)',
  'of',
  'First',
  'Class',
  'Technical',
  'Machinery',
  'Sodapacked',
  'in',
  'blue',
  'crates',
  'labelled',
  "'Test",
  'Machine',
  "London'",
  'Unit',
  'Price',
  'Usd',
  '50',
  'DELIVERY',
  'TERMS;',
  'CIF',
  'TORONTO',
  'HS',
  'Code',
  '55678334'],
 'GoodsDescription': [],
 'Incoterms': ['+/-'],
 'GoodsOrigin': [],
 'Tolerance': [],
 'HSCode': []}